In [2]:
import pickle
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances

In [11]:
!head -n 1 data_filtered.csv


,text,industry,cluster


In [3]:
data = pd.read_csv('data_filtered.csv')
data.head(10)

,Unnamed: 0,text,industry,cluster
0,0,benefits board benefits board homejoin plannin...,marketing and advertising,3
1,1,home columbia landcare environmental stewardsh...,environmental services,4
2,2,region inc worldwide revenue accelerating worl...,management consulting,2
3,3,home team careers services civil design case s...,civil engineering,4
4,4,home friedman associates friedman associates c...,accounting,2
5,5,big rapids real estate search houses homes sal...,real estate,3
6,6,garcia law firm garcia law firm home attorney ...,legal services,2
7,7,advfn free stock cryptocurrency prices charts ...,internet,3
8,8,guardian athletics kato collar invest online s...,sports,3
9,9,miracle doors america premier door service com...,utilities,4


In [4]:
with open('vectors.pkl', 'rb') as f:
    vectors = pickle.load(f)

vectors.shape

(1694586, 8088615)

In [13]:
vectors[[1,4,5]]

<3x8088615 sparse matrix of type '<class 'numpy.float64'>'
	with 632 stored elements in Compressed Sparse Row format>

In [6]:
industries = data.industry.unique()
clusters = data.cluster.unique()

distance = {}

for i in industries:
    for c in clusters:
        print(i,c)
        left = data[(data.industry == i) & (data.cluster == c)]
        if left.shape[0] > 0:
            right = data[(data.industry != i) & (data.cluster == c)]
            assert right.shape[0] > 0

            for 
            # dist = pairwise_distances(vectors[left.index], vectors[right.index], metric='cosine')
        break
    break

marketing and advertising 3


In [7]:
left

,Unnamed: 0,text,industry,cluster
0,0,benefits board benefits board homejoin plannin...,marketing and advertising,3
207,207,green card wisconsin newest discount golf card...,marketing and advertising,3
214,214,home cubed site homeabout ushow worksrequest d...,marketing and advertising,3
317,317,gauger associates case studies capabilities cl...,marketing and advertising,3
365,365,bloc influencers agency bloc services influenc...,marketing and advertising,3
...,...,...,...,...
1694246,1694246,shipley brown media group home media solutions...,marketing and advertising,3
1694309,1694309,box home courses tob teams testimonials jobs b...,marketing and advertising,3
1694372,1694372,fms expert management advisory services field ...,marketing and advertising,3
1694482,1694482,welcome professional manufacturers pmr toggle ...,marketing and advertising,3


In [8]:
right

,Unnamed: 0,text,industry,cluster
5,5,big rapids real estate search houses homes sal...,real estate,3
7,7,advfn free stock cryptocurrency prices charts ...,internet,3
8,8,guardian athletics kato collar invest online s...,sports,3
10,10,valerion biotechnology company focused develop...,biotechnology,3
11,11,test page latticesystems com html body div spa...,e-learning,3
...,...,...,...,...
1694577,1694577,warehouse management systems software wms syst...,computer software,3
1694581,1694581,home byron bethany irrigation district menu ho...,government administration,3
1694582,1694582,official online store ewing athletics ewing at...,apparel & fashion,3
1694583,1694583,staffing undercover productions las vegas unde...,entertainment,3


In [14]:
from itertools import combinations

In [1]:
to_compute = {}

for i in left.index:
    for j in right.index:
        to_compute[(i,j)] = 0

to_compute

NameError: name 'left' is not defined

In [ ]:
to_compute

In [ ]:
cosine_values = pairwise_distances(X=tfidf_industry, Y=tfidf_other, metric='cosine')
cosine_values = scipy.sparse.csr_matrix(cosine_values)
indices = cosine_values.argmax(axis=1)
for i, index in enumerate(current_industry_indices):
    related_of[index] = current_other_indices[indices[i,0]]
print(cosine_values)